In [45]:
import os
import openpyxl
import numpy as np
import pandas as pd
import matplotlib as plt

**Basic info** <br>
 <br>
After printing the info of the dataframe, we can know the dimension is [3645 rows x 56 columns], which tells us there are 3645 trades, spanning from 2000-01-03 to 2014-06-30.

In [39]:
# Read in the excel as a dataframe
data = pd.read_excel("Assignment_PAIRS_data.xlsx", sheet_name='data', header=6)

### Calculate Returns
Now we are going to calculate the **returns** for both White and Black.

In [51]:
def calculate_returns(df, strat, period):
    ret = (df[f"close{strat}"] - df[f"close{strat}"].shift(period)) / df[f"close{strat}"].shift(period)
    return ret

data["ret1WHITE"] = calculate_returns(data, "WHITE", 1)
data["ret5WHITE"] = calculate_returns(data, "WHITE", 5)
data["ret10WHITE"] = calculate_returns(data, "WHITE", 10)
data["ret20WHITE"] = calculate_returns(data, "WHITE", 20)

data["ret1BLACK"] = calculate_returns(data, "BLACK", 1)
data["ret5BLACK"] = calculate_returns(data, "BLACK", 5)
data["ret10BLACK"] = calculate_returns(data, "BLACK", 10)
data["ret20BLACK"] = calculate_returns(data, "BLACK", 20)

print(data["ret10WHITE"])

0            NaN
1            NaN
2            NaN
3            NaN
4            NaN
          ...   
3640   -0.004185
3641    0.009114
3642    0.015937
3643    0.018007
3644    0.020445
Name: ret10WHITE, Length: 3645, dtype: float64


### Calculate volatility

In [52]:
daily_returns_white = data["closeWHITE"].pct_change().dropna()
data["vol20WHITE"] = daily_returns_white[-20:].std()

data["vol20WHITE"]

0       0.007105
1       0.007105
2       0.007105
3       0.007105
4       0.007105
          ...   
3640    0.007105
3641    0.007105
3642    0.007105
3643    0.007105
3644    0.007105
Name: vol20WHITE, Length: 3645, dtype: float64

### Calculate Z-scores

In [42]:
def calculate_z(df, strat, period):
    # df['last_value'] = df['your_column_name'].shift(1)
    zret = (df[f"ret{period}{strat}"] - df[f"ret{period}{strat}"].mean()) / df[f"ret{period}{strat}"].std()
    return zret

periods = [5,10,20]
for i in periods:
    data[f"zWHITE{i}"] = calculate_z(data, "WHITE", i)

for i in periods:
    data[f"zBLACK{i}"] = calculate_z(data, "BLACK", i)

print(data["zBLACK10"])

0      -0.069411
1      -0.069411
2      -0.069411
3      -0.069411
4      -0.069411
          ...   
3640    0.609055
3641    0.812387
3642    0.944424
3643    1.078076
3644    0.994780
Name: zBLACK10, Length: 3645, dtype: float64


### Calculate zdiff

In [44]:
data["zdiff5"] = data["zWHITE5"] - data["zBLACK5"]
data["zdiff10"] = data["zWHITE10"] - data["zBLACK10"]
data["zdiff10"] = data["zWHITE10"] - data["zBLACK10"]
